In [1]:
from concurrent.futures import ThreadPoolExecutor
import time
import threading

class CustomSum:
    def  __init__(self):
        self.value = 0

    def  update(self, n):
        print(n)
        tmp_value = self.value
        tmp_value += 1
        time.sleep(0.1)
        self.value = tmp_value

cs = CustomSum()
with ThreadPoolExecutor(max_workers=3)  as executor:
    for n in  ["first",  "second",  "third"]:
        executor.submit(cs.update, n)
print(cs.value)

first
second
third
1


In [5]:
from concurrent.futures import ThreadPoolExecutor
import time
import threading

class CustomSum:
    def  __init__(self):
        self.value = 0
        self._lock = threading.Lock()

    def  update(self, n):
        print(n)
        self._lock.acquire()
        tmp_value = self.value
        tmp_value += 1
        time.sleep(0.1)
        self.value = tmp_value
        self._lock.release()

cs = CustomSum()
with ThreadPoolExecutor(max_workers=3)  as executor:
  for n in  ["first",  "second",  "third"]:
      executor.submit(cs.update, n)

print(cs.value)

first
second
third
3


In [3]:
from concurrent.futures import ThreadPoolExecutor
import time
import threading

class CustomSum:
    def  __init__(self):
        self.value = 0
        self._lock = threading.Lock()

    def  update(self, n):
        print(n)
        with  self._lock:
            tmp_value = self.value
            tmp_value += 1
            time.sleep(0.1)
            self.value = tmp_value

cs = CustomSum()
start_time = time.time()
with ThreadPoolExecutor(max_workers=3)  as executor:
    for n in  ["first",  "second",  "third"]:
        executor.submit(cs.update, n)
print(time.time()-start_time)
print(cs.value)

firstsecond

third
0.3040354251861572
3


In [6]:
import threading
import time

class CustomSum:
    def  __init__(self):
        self.value = 0
        self._sem = threading.Semaphore(1)

    def  update(self, n):
        print(n)
        with  self._sem:
            tmp_value = self.value
            tmp_value += 1
            time.sleep(0.1)
            self.value = tmp_value

cs = CustomSum()
start_time = time.time()
with ThreadPoolExecutor(max_workers=3)  as executor:
    for n in  ["first",  "second",  "third"]:
      executor.submit(cs.update, n)
print(time.time()-start_time)
print(cs.value)

first
second
third
0.30527448654174805
3


In [8]:
import concurrent.futures
import queue
import random
import threading
import time

def  send(queue, event):
    while  not event.is_set():
      message = random.randint(1,  11)
      queue.put(message)
      print(f'send : {message}')
    print("send terminate")

def  bucket(queue, event):
    while  not event.is_set()  or  not queue.empty():
      try:
          message = queue.get(timeout=1)
          print(f'bucket : {message}')
      except queue.Empty:
          print("empty")
          pass
    print("bucket terminate")

pipeline = queue.Queue(maxsize=10)
event = threading.Event()
with concurrent.futures.ThreadPoolExecutor(max_workers=2)  as executor:
    executor.submit(send, pipeline, event)
    executor.submit(bucket, pipeline, event)
    time.sleep(0.1)
    event.set()

send : 2bucket : 2

send : 2bucket : 2

send : 2bucket : 2

send : 9bucket : 9

send : 1bucket : 1

send : 1
send : 6
send : 7
send : 11
send : 9
send : 2
send : 4
send : 9
send : 4
send : 7
bucket : 1
bucket : 6
bucket : 7
bucket : 11
bucket : 9
bucket : 2
bucket : 4
bucket : 9
bucket : 4
bucket : 7
send : 2
send : 2
send : 9
send : 8
send : 8
send : 2
send : 2
send : 4
send : 6
send : 10
bucket : 2
bucket : 2
bucket : 9
bucket : 8
bucket : 8
bucket : 2
bucket : 2
bucket : 4
bucket : 6
bucket : 10
send : 5
send : 9
send : 7
send : 4
send : 1
send : 6
send : 1
send : 10
send : 5
send : 9
bucket : 5
bucket : 9
bucket : 7
bucket : 4
bucket : 1
bucket : 6
bucket : 1
bucket : 10
bucket : 5
bucket : 9
send : 6
send : 3
send : 7
send : 8
send : 8
send : 3
send : 2
send : 9
send : 7
send : 5
bucket : 6send : 7

bucket : 3
bucket : 7
bucket : 8
bucket : 8send : 8
send : 9
send : 2
send : 8

bucket : 3
bucket : 2
bucket : 9
bucket : 7
bucket : 5
bucket : 7
bucket : 8
bucket : 9
bucket : 2
bucke